In [1]:
print("Ganpati Bappa Morya")

Ganpati Bappa Morya


In [2]:
import numpy as np
import pandas as pd
import nltk
import string
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score

In [3]:
from nltk.corpus import stopwords
stopwords.words('english')

['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each

In [4]:
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
ps.stem('loving')

'love'

In [5]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to C:\Users\Vedang
[nltk_data]     Chikane\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [6]:
raw_mail_data = pd.read_csv('spam.csv')

In [7]:
print(raw_mail_data)

     Category                                            Message
0         ham  Go until jurong point, crazy.. Available only ...
1         ham                      Ok lar... Joking wif u oni...
2        spam  Free entry in 2 a wkly comp to win FA Cup fina...
3         ham  U dun say so early hor... U c already then say...
4         ham  Nah I don't think he goes to usf, he lives aro...
...       ...                                                ...
5768     spam  Subject: business proposal .\nfrom : alhaji su...
5769     spam  Subject: transfer manager vacancy\nsecondary h...
5770     spam  Subject: important online banking alert\ndear ...
5771     spam  Subject: here u can buy soft . ware at che - a...
5772     spam  Subject: cartier , piaget , rolex replicas - s...

[5773 rows x 2 columns]


In [8]:
df = raw_mail_data.where((pd.notnull(raw_mail_data)),'')

In [9]:
df.head()

,Category,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [10]:
df.shape

(5773, 2)

In [11]:
# label spam mail as 0;  ham mail as 1;

df.loc[df['Category'] == 'spam', 'Category',] = 0
df.loc[df['Category'] == 'ham', 'Category',] = 1

In [12]:
# separating the data as texts and label

Y = df['Category']

In [13]:
def transform_text(Message):
    Message = Message.lower()
    Message = nltk.word_tokenize(Message)
    
    Y = []
    for i in Message:
        if i.isalnum():
            Y.append(i)
    
    Message = Y[:]
    Y.clear()
    
    for i in Message:
        if i not in stopwords.words('english') and i not in string.punctuation:
            Y.append(i)
            
    Message = Y[:]
    Y.clear()
    
    for i in Message:
        Y.append(ps.stem(i))
    
            
    return " ".join(Y)

In [14]:
transform_text("I'm gonna be home soon and i don't want to talk about this stuff anymore tonight, k? I've cried enough today.")

'gon na home soon want talk stuff anymor tonight k cri enough today'

In [15]:
df['transformed_text'] = df['Message'].apply(transform_text)

In [16]:
df.head()

,Category,Message,transformed_text
0,1,"Go until jurong point, crazy.. Available only ...",go jurong point crazi avail bugi n great world...
1,1,Ok lar... Joking wif u oni...,ok lar joke wif u oni
2,0,Free entry in 2 a wkly comp to win FA Cup fina...,free entri 2 wkli comp win fa cup final tkt 21...
3,1,U dun say so early hor... U c already then say...,u dun say earli hor u c alreadi say
4,1,"Nah I don't think he goes to usf, he lives aro...",nah think goe usf live around though


In [17]:
print(Y)

0       1
1       1
2       0
3       1
4       1
       ..
5768    0
5769    0
5770    0
5771    0
5772    0
Name: Category, Length: 5773, dtype: object


In [18]:
df.shape

(5773, 3)

In [19]:
tfidf =TfidfVectorizer(stop_words='english', max_df=0.85,min_df=2,ngram_range=(1, 2))

In [20]:
X = tfidf.fit_transform(df['transformed_text']).toarray()


In [21]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=3)

In [22]:
print(X.shape)
print(X_train.shape)
print(X_test.shape)

(5773, 9159)
(4618, 9159)
(1155, 9159)


In [23]:
# convert Y_train and Y_test values as integers

Y_train = Y_train.astype('int')
Y_test = Y_test.astype('int')

In [24]:
from sklearn.svm import SVC

# Create an instance of the SVM classifier
svm_classifier = SVC(kernel='sigmoid')  # You can specify the kernel type you want (e.g., 'linear', 'rbf', 'poly', etc.)

# Fit the SVM classifier to your training data
svm_classifier.fit(X_train, Y_train)  # X_train is your training data, y_train is your training labels

# Make predictions using the trained SVM classifier
y_pred = svm_classifier.predict(X_test)  # X_test is your test data

# Evaluate the SVM classifier (e.g., calculate accuracy, precision, recall, etc.)


In [25]:

accuracy = accuracy_score(Y_test, y_pred)

print(f"Accuracy: {accuracy:.2f}")


Accuracy: 0.98


In [26]:
precision = precision_score(Y_test, y_pred)

print(f"Precision: {precision:.2f}")

Precision: 0.98


In [27]:
import pickle
pickle.dump(tfidf,open('vectorizer.pkl','wb'))
pickle.dump(svm_classifier,open('model.pkl','wb'))

In [28]:
input_mail = ["I'm gonna be home soon and i don't want to talk about this stuff anymore tonight, k? I've cried enough today."]
input_data_features = tfidf.transform(input_mail)
#Load your trained SVM model
svm_model = SVC(kernel='sigmoid')  

from joblib import load
# Load your trained SVM model
svm_model = load('model.pkl')  # Load your trained model file
input_data_features_dense = input_data_features.toarray()

# Make predictions
prediction = svm_model.predict(input_data_features_dense)
if prediction[0] == 1:
    print('Ham mail')
else:
    print('Spam mail')



Ham mail
